In [97]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
# algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import lightgbm as ltgbm
from collections import Counter

In [78]:
data = pd.read_csv("train_s3TEQDk.csv")
test = pd.read_csv("test_mSzZ8RL.csv")

In [79]:
y_train = data['Is_Lead']
x_train = data.drop('Is_Lead',axis = 1)
x_test = test.copy()

In [80]:
new_dataset = pd.concat([x_train,x_test])

In [81]:
new_dataset = new_dataset.drop('ID',axis = 1)

In [82]:
new_dataset.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,Female,73,RG268,Other,X3,43,No,1045696,No
1,Female,30,RG277,Salaried,X1,32,No,581988,No
2,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes
3,Male,34,RG270,Salaried,X1,19,No,470454,No
4,Female,30,RG282,Salaried,X1,33,No,886787,No


In [83]:
list_columns = ['Gender','Region_Code','Occupation','Channel_Code','Is_Active']

In [84]:
#filling null values
label = LabelEncoder()
fill_null = new_dataset.copy()
fill_null_y_train = fill_null['Credit_Product'][fill_null['Credit_Product'].isnull()!=True]
fill_null_x_train = fill_null[fill_null['Credit_Product'].isnull()!= True]
fill_null_x_train.drop('Credit_Product',axis = 1,inplace = True)
fill_null_test = fill_null[fill_null['Credit_Product'].isnull() == True]
fill_null_test.drop('Credit_Product',axis = 1,inplace = True)
for i in list_columns:
    fill_null_x_train[i] = label.fit_transform(fill_null_x_train[i])
    fill_null_test[i] = label.fit_transform(fill_null_test[i])
fill_null_y_train = label.fit_transform(fill_null_y_train)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [85]:
fill_null_test

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Avg_Account_Balance,Is_Active
6,1,62,32,1,2,20,1056750,1
15,1,33,18,3,1,69,517063,1
31,0,32,29,2,3,15,1072850,1
36,0,41,18,3,2,62,962383,0
40,0,63,4,1,2,103,1249319,1
...,...,...,...,...,...,...,...,...
105260,1,45,20,2,2,117,582015,1
105266,1,46,6,3,1,86,488022,1
105272,1,48,18,3,1,91,1228271,1
105286,1,58,27,3,2,80,750421,1


In [86]:
knn = KNeighborsClassifier()
knn.fit(fill_null_x_train,fill_null_y_train)
list_credit = knn.predict(fill_null_test)
list_credit = label.inverse_transform(list_credit)
new_dataset['Credit_Product'][new_dataset['Credit_Product'].isnull()==True] = list_credit.reshape(1,-1)
new_dataset.isnull().sum()

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Gender                 0
Age                    0
Region_Code            0
Occupation             0
Channel_Code           0
Vintage                0
Credit_Product         0
Avg_Account_Balance    0
Is_Active              0
dtype: int64

In [87]:
new_dataset['Credit_Product'].value_counts()

No     235186
Yes    115851
Name: Credit_Product, dtype: int64

In [88]:
encode_column = ['Gender','Occupation','Channel_Code','Credit_Product','Is_Active']
for i in encode_column:
    dummy = pd.get_dummies(new_dataset[i],prefix = i,drop_first = True)
    new_dataset.drop(i,axis = 1,inplace = True)
    new_dataset = pd.concat([new_dataset,dummy],axis = 1)

In [91]:
new_dataset

,Age,Region_Code,Vintage,Avg_Account_Balance,Gender_Male,Occupation_Other,Occupation_Salaried,Occupation_Self_Employed,Channel_Code_X2,Channel_Code_X3,Channel_Code_X4,Credit_Product_Yes,Is_Active_Yes
0,73,18,43,1045696,0,1,0,0,0,1,0,0,0
1,30,27,32,581988,0,0,1,0,0,0,0,0,0
2,56,18,26,1484315,0,0,0,1,0,1,0,0,1
3,34,20,19,470454,1,0,1,0,0,0,0,0,0
4,30,32,33,886787,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105307,52,18,86,4242558,1,0,1,0,1,0,0,1,1
105308,55,27,86,1159153,1,1,0,0,1,0,0,1,0
105309,35,4,15,1703727,1,0,1,0,0,0,1,0,0
105310,53,4,93,737178,1,1,0,0,0,1,0,0,1


In [90]:
new_dataset['Region_Code'] = label.fit_transform(new_dataset['Region_Code'])

In [92]:
# split the train and test data
x_train = new_dataset.iloc[:len(data),:]
validate_file = new_dataset.iloc[len(data):,:]

In [93]:
y_train = label.fit_transform(data['Is_Lead'])

In [94]:
#oversampling
x,y = SMOTE().fit_resample(x_train,y_train)

In [95]:
x.shape,y.shape

((374874, 13), (374874,))

In [98]:
#training and testing the model with train data
x_train,x_test,y_train,y_test = train_test_split(x,y)
log = LogisticRegression()
log.fit(x_train,y_train)
print("logistic_regression: {}".format(roc_auc_score(y_test,log.predict_proba(x_test)[:,1])))

rf = RandomForestClassifier()
rf.fit(x_train,y_train)
print("RF_classifier: {}".format(roc_auc_score(y_test,rf.predict_proba(x_test)[:,1])))

gb = GradientBoostingClassifier()
gb.fit(x_train,y_train)
print("Gradient_boost_classifier: {}".format(roc_auc_score(y_test,gb.predict_proba(x_test)[:,1])))

XGB = XGBClassifier()
XGB.fit(x_train,y_train)
print("XGboost_classifier: {}".format(roc_auc_score(y_test,XGB.predict_proba(x_test)[:,1])))

df = ltgbm.Dataset(x_train,label = y_train)
params = {'learning_rate' : 0.01,'boosting_type':'gbdt','objective' : 'binary', 'metric':['auc','binary_logloss'],
         'num_leaves':100,'max_depth':10}
model = ltgbm.train(params,df,500)
print("lightgbm_classifier: {}".format(roc_auc_score(y_test,model.predict(x_test))))

logistic_regression: 0.5554043822545388
RF_classifier: 0.9030411568870267
Gradient_boost_classifier: 0.896690814400495


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:58:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGboost_classifier: 0.9150089793838533
[LightGBM] [Info] Number of positive: 140536, number of negative: 140619
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011420 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 498
[LightGBM] [Info] Number of data points in the train set: 281155, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499852 -> initscore=-0.000590
[LightGBM] [Info] Start training from score -0.000590
lightgbm_classifier: 0.9140364510184819
